In [ ]:
%matplotlib inline


Leave One Out Per Class (LOOPC)
======================================================

This example shows how to make a Leave One Out for each class.




Import librairies
-------------------------------------------



In [ ]:
from museotoolbox.cross_validation import LeaveOneOutPerClass
from museotoolbox import datasets

Load HistoricalMap dataset
-------------------------------------------



In [ ]:
X,y = datasets.getHistoricalMap(return_X_y=True)

Create CV
-------------------------------------------



In [ ]:
LOOPC = LeaveOneOutPerClass(random_state=8,verbose=False)
for tr,vl in LOOPC.split(X=None,y=y):
    print(tr,vl)

<div class="alert alert-info"><h4>Note</h4><p>Split is made to generate each fold</p></div>



In [ ]:
# Show label

for tr,vl in LOOPC.split(X=None,y=y):
    print(y[vl])